In [12]:
import pandas as pd
import numpy as np
import time
import requests
import re

In [13]:
# Helper functions
def timer_function(func=None, n_run=1):

    ti = time.time()
    for n in range(n_run):
        func
    tf = time.time()
    
    print(f'Runtime: {(tf - ti)/n_run} s')

def read_input_pd(file_path=None):
    df_data = pd.read_csv(file_path, sep=',', header=None)
    
    return df_data

def read_input_lst(file_path=None):
    with open(file_path,'r') as f:
        lines = f.read().splitlines()

    input_arr = list(lines)


    return input_arr

def scrape_data(day):
    """
    fetches the input data from the website
    day: the day of december to fetch
    """
    # Fetch session cookie value from local file
    with open("session.txt",'r') as f:
        key = f.read()
    cookie = {'session': key}
    url= f"https://adventofcode.com/2021/day/{day}/input"

    with requests.Session() as s:
        r = s.get(url, cookies = cookie).text
        # Data always end with a blanc line. Data is split on line change
        input_lst = re.split('\n', r)[:-1]

    return input_lst

timer_function(func=read_input_pd("input/test_csv.csv"), n_run=10)
print(read_input_lst("input/test_csv.csv"))
print(read_input_pd("input/test_csv.csv"))
print(scrape_data(day=1)[:10])


Runtime: 0.0 s
['1,a,mdksa sm', '2,b, jdsasjnj203', '3,c, dj1992']
   0  1             2
0  1  a      mdksa sm
1  2  b   jdsasjnj203
2  3  c        dj1992
['159', '158', '174', '196', '197', '194', '209', '213', '214', '222']


# December 1


In [14]:
# Test data
input_test = np.array([199,200,208,210,200,207,240,269,260,263])

def dec1_part1(data):
    answer = np.sum(np.diff(data) > 0)
    return answer

# Test data
print(f'Test input:\n', input_test)
print(f'Solution test: ', dec1_part1(input_test))

Test input:
 [199 200 208 210 200 207 240 269 260 263]
Solution test:  7


In [15]:
# Actual input
input_1 = scrape_data(day=1)
input_1 = [int(i) for i in input_1]
answer = dec1_part1(input_1)
print(f'Solution december 1. - part 1: ', answer)
timer_function(func=dec1_part1(input_1), n_run=1000000)

Solution december 1. - part 1:  1583
Runtime: 1.5630483627319337e-08 s


In [16]:
# Part 2 Test
def dec1_part2(data, window):
    rolling_sum =[np.sum(data[i:i + window]) for i in range(0, len(data) + 1 - window)]
    answer = dec1_part1(rolling_sum)

    return rolling_sum, answer

# Test input
rolling_sum,answer = dec1_part2(input_test, window=3)
print(f'Sums:', rolling_sum)
print(f'Solution test: ', answer)

Sums: [607, 618, 618, 617, 647, 716, 769, 792]
Solution test:  5


In [17]:
# Actual input
rolling_sum, answer = dec1_part2(input_1, window=3)
print(f'Solution december 1. - part 2: ', answer)
timer_function(func=dec1_part2(input_1, window=3), n_run=1000000)

Solution december 1. - part 2:  1627
Runtime: 1.5645265579223633e-08 s


# December 2


In [18]:
input_test = ['forward 5','down 5','forward 8','up 3','down 8','forward 2']
input_2 = scrape_data(day=2)
print('Number of input rows:', len(input_2))

Number of input rows: 1000


In [19]:
# Solution and test input
def split_direction(df, direction, sign):
    mask = df['raw'].str.contains(direction)
    df.loc[mask, 'direction'] = direction[0]
    df.loc[mask, 'step'] = df.loc[mask, 'raw'].str.replace(direction, '').astype(int)*sign
    
    return df

def dec2_part1(data):
    df_data = pd.DataFrame(data).rename(columns={0:'raw'})
    
    df_data = split_direction(df_data, 'forward ', +1)
    df_data = split_direction(df_data, 'down ', +1)
    df_data = split_direction(df_data, 'up ', -1)
    
    mask_fwd = df_data['direction'] == 'f'
    pos_horizontal = df_data.loc[mask_fwd, 'step'].sum()
    pos_vertical = df_data.loc[~mask_fwd, 'step'].sum()
    
    answer = pos_horizontal*pos_vertical
    
    return pos_horizontal, pos_vertical, df_data, answer


pos_horizontal, pos_vertical, df_test ,answer = dec2_part1(input_test)
print(df_test, '\n')
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution test: ', answer)

         raw direction  step
0  forward 5         f   5.0
1     down 5         d   5.0
2  forward 8         f   8.0
3       up 3         u  -3.0
4     down 8         d   8.0
5  forward 2         f   2.0 

Position (horizontal/vertical): 15.0 , 10.0
Solution test:  150.0


In [20]:
# Actual input solution
pos_horizontal, pos_vertical, df_input, answer = dec2_part1(input_2)
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution: ', answer)
timer_function(func=dec2_part1(input_2), n_run=100000)

Position (horizontal/vertical): 1939.0 , 1109.0
Solution:  2150351.0
Runtime: 0.0 s


In [21]:
# December  2 - Part 2
def dec2_part2(df):    
    mask_fwd = df['direction'] == 'f'
    
    df['aim_step'] = 0
    df.loc[~mask_fwd, 'aim_step'] = df.loc[~mask_fwd, 'step']
    df['aim'] = df['aim_step'].cumsum()

    pos_horizontal = df.loc[mask_fwd, 'step'].sum()
    pos_vertical = (df.loc[mask_fwd, 'step']*df.loc[mask_fwd, 'aim']).sum()
    
    answer = pos_horizontal*pos_vertical
    
    return pos_horizontal, pos_vertical, df, answer

pos_horizontal, pos_vertical, df_test, answer = dec2_part2(df_test)
print(df_test, '\n')
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution test: ', answer)

         raw direction  step  aim_step   aim
0  forward 5         f   5.0       0.0   0.0
1     down 5         d   5.0       5.0   5.0
2  forward 8         f   8.0       0.0   5.0
3       up 3         u  -3.0      -3.0   2.0
4     down 8         d   8.0       8.0  10.0
5  forward 2         f   2.0       0.0  10.0 

Position (horizontal/vertical): 15.0 , 60.0
Solution test:  900.0


In [22]:
pos_horizontal, pos_vertical, df_input, answer = dec2_part2(df_input)
print(f'Position (horizontal/vertical):', pos_horizontal, ',',pos_vertical)
print(f'Solution: ', answer)
timer_function(func=dec2_part2(df_input), n_run=10000000)

Position (horizontal/vertical): 1939.0 , 950357.0
Solution:  1842742223.0
Runtime: 1.5365934371948243e-08 s


In [23]:
# december 2 part 2 - non pandas
input_test
input_ = [()]

# December 3

In [24]:
input_test = ['00100','11110','10110','10111','10101','01111','00111','11100','10000','11001','00010','01010']
input_3 = scrape_data(day=3)
print(input_3[:10])
print('Number of input rows:', len(input_3))


['011001101000', '010101111100', '000000111101', '001001001010', '010011000001', '111111001110', '001011101111', '110011010110', '100011111110', '101110000111']
Number of input rows: 1000


In [25]:
# Solution part 1 and test
def dec3_part1(data):
    n_digit = len(data[0])
    n_row = len(data)
    
    digit_sum = np.zeros(n_digit)
    
    for row in data:
        row_lst = [int(i) for i in row]
        digit_sum = np.add(digit_sum, row_lst)
    
    mask = digit_sum >= n_row/2
    gamma = ''.join(map(str, 1*(mask)))
    epsilon = ''.join(map(str, 1*(~mask)))
    
    answer = int(gamma,2)*int(epsilon,2)
    
    return gamma, epsilon, answer
    
print(dec3_part1(input_3))
timer_function(func=dec3_part1(input_3), n_run=1000000)

('010001110111', '101110001000', 3374136)
Runtime: 1.5612363815307617e-08 s


In [44]:
def find_rating(rating, bit):
    n_digit = len(rating[0])
    n_row = len(rating)
    idx = 0
    
    while (len(rating) > 1) & (idx < n_digit):
        digit_sum = 0
        for row in rating:
            digit_sum += int(row[idx])
            
        digit_even = digit_sum == len(rating)/2
        
        if digit_even:
            rating = [entry for entry in rating if entry[idx] == bit]
        else:
            if bit == '0':
                digit_match = 1*(np.mean(digit_sum) < 1)
            else:
                digit_match = 1*(np.mean(digit_sum) > 1)

            rating = [entry for entry in rating if entry[idx] == str(digit_match)]

        idx += 1
    
    # select the only remaining correct entry if more exist
    rating = [entry for entry in rating if entry[-1] == bit][0]
    
    return rating

def dec3_part2(data):
    o_rating = find_rating(rating=data, bit='1')
    co2_rating = find_rating(rating=data, bit='0')
        
    answer = int(o_rating,2)*int(co2_rating,2)
    
    return o_rating, co2_rating, answer

print(dec3_part2(input_3))
timer_function(func=dec3_part2(input_3), n_run=1000000)

IndexError: list index out of range